In [90]:
import duckdb as db
import pandas as pd
import numpy as np

def sql(query, database = "../data/database.duckdb"):
    con = db.connect(database)
    res = con.execute(query).df()
    con.close()
    return res

In [52]:
#Conversion de notre base de donnée acquise via SQL en DataFrame
data = pd.DataFrame(sql("""
    SELECT
        *
    FROM
       dataframe_view
    """))

ANALYSE EXPLORATOIRE DES DONNEES

In [53]:
data.isna().sum()

tconst                     0
Title                      0
Genres                     0
Year                       0
Runtime                    0
Actors                     0
Director                   0
Writers                 2151
Resume                  1151
Rating                     0
Votes                      0
Budget                     0
Revenue                    0
Profit                     0
ProductionCompanies        0
ProductionCountries    60111
Poster                  2231
dtype: int64

In [54]:
#Creation d'un DF_Copy pour ne pas modifier le DF d'origine
data_copy = data

In [84]:
data_copy.sample(2)

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
50554,tt0116341,Forest Warrior,"Action,Adventure,Comedy",1996,93,"William Sanderson, Jordan Brower, Michael Frie...",Aaron Norris,"Galen Thompson, Ron Swanson",John McKenna is a spiritual being who is able ...,3.6,3037,0,0,0,"['Nu Image', 'LOT Productions']",[Unknown],/lU9ECdGWnhWl2pJKlovMSWQKmLB.jpg
86638,tt7106968,Ride,Drama,2018,95,"Silvia Gallerano, Renato Carpentieri, Chiara M...",Valerio Mastandrea,"Valerio Mastandrea, Enrico Audenino","A woman and her 10-year-old son face, in their...",6.0,368,0,0,0,"['Kimerafilm', 'RAI Cinema']","['', 'IT']",/qcE96LRqOg3lLySIziJKF17aJwU.jpg


In [14]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101563 entries, 0 to 101562
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   tconst               101563 non-null  object 
 1   Title                101563 non-null  object 
 2   Genres               101563 non-null  object 
 3   Year                 101563 non-null  object 
 4   Runtime              101563 non-null  object 
 5   Actors               101563 non-null  object 
 6   Director             101563 non-null  object 
 7   Writers              99412 non-null   object 
 8   Resume               100412 non-null  object 
 9   Rating               101563 non-null  float64
 10  Votes                101563 non-null  int64  
 11  Budget               101563 non-null  int64  
 12  Revenue              101563 non-null  int64  
 13  Profit               101563 non-null  int64  
 14  ProductionCompanies  101563 non-null  object 
 15  ProductionCountri

In [58]:
#60111 valeurs manquantes dans la colonne ProductionCountries
data_copy['ProductionCountries'].isna().sum()
#On remplace les valeurs manquantes par une ['Unknown'] pour garder la structure de la colonne
data_copy['ProductionCountries'] = data_copy['ProductionCountries'].apply(lambda x: ['Unknown'] if pd.isna(x) else x)

In [67]:
#Remplacer les valeurs manquantes des colonnes Writers, Resume, Poster 
data_copy['Writers'] = data_copy['Writers'].apply(lambda x: ['Unknown'] if pd.isna(x) else x)
data_copy['Resume'] = data_copy['Resume'].apply(lambda x: ['Unknown'] if pd.isna(x) else x)
data_copy['Poster'] = data_copy['Poster'].apply(lambda x: ['Unknown'] if pd.isna(x) else x)

In [86]:
data_copy['Resume'].sample(10)

70466    The mechanic Jack tries to get rid of his crim...
19950    Superstar Manoranjan learns that he has only a...
23479    The Olsen Gang realizes that cash is obsolete ...
16581    Orvar is a vagabond, walking the country roads...
25708    A series of murders strikes the city of Fentun...
81139    The story is narrated by ten-year old Staszek,...
93363    Fifty Shades Uncovered is a brand new feature ...
83801    A Civil War guerilla gang plans an attack on a...
46027    Finding Happiness is a film that captures a jo...
58251    The buoyant Molly Brown has survived the first...
Name: Resume, dtype: object

PREMIER ESSAI d'UN KNN SUR YEAR POUR ESSAYER L'ALGO

In [88]:
#import de la bibliothèque scikit-learn
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



In [99]:
data_copy[data_copy['Year'].str.contains('\\\\N')]

,tconst,Title,Genres,Year,Runtime,Actors,Director,Writers,Resume,Rating,Votes,Budget,Revenue,Profit,ProductionCompanies,ProductionCountries,Poster
20287,tt2075291,Cyborg Nemesis: The Dark Rift,"Action,Sci-Fi",\N,100,"Cazzy Golomb, Vincent Klyn, Terrie Batson, Sas...",Albert Pyun,Cynthia Curnan,A U.S. Marine special ops team awakens from hy...,6.4,122,0,0,0,[],[],/hiGNRlOk956lvxIBV6dZ5rJGn0l.jpg


In [103]:
#nettoyage de la colonne Year (il semblerait qu'il y ait qu'une ligne avec une valeur '\N')
data_copy['Year'] = data_copy['Year'].replace('\\N', np.nan)
data_copy = data_copy.dropna(subset=['Year'])
#On convertit la colonne Year en int pour pouvoir l'utiliser dans le modèle
data_copy['Year'] = data_copy['Year'].astype(int)
#On vérifie que la colonne Year ne contient plus de valeurs manquantes
data_copy.info()


<class 'pandas.core.frame.DataFrame'>
Index: 101562 entries, 0 to 101562
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   tconst               101562 non-null  object 
 1   Title                101562 non-null  object 
 2   Genres               101562 non-null  object 
 3   Year                 101562 non-null  int64  
 4   Runtime              101562 non-null  object 
 5   Actors               101562 non-null  object 
 6   Director             101562 non-null  object 
 7   Writers              101562 non-null  object 
 8   Resume               101562 non-null  object 
 9   Rating               101562 non-null  float64
 10  Votes                101562 non-null  int64  
 11  Budget               101562 non-null  int64  
 12  Revenue              101562 non-null  int64  
 13  Profit               101562 non-null  int64  
 14  ProductionCompanies  101562 non-null  object 
 15  ProductionCountries  1

In [105]:
#Tentative de KNN sur Year
X = data_copy[['Year']]

knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X)

distances, indices = knn.kneighbors(X.iloc[[0]])
print("Distances:", distances)
print("Indices:", indices)

Distances: [[0. 0. 0. 0. 0.]]
Indices: [[ 57 101  97   0  84]]


les résultats indiquent que le knn fonctionnent mais ne sert à rien puisque basé que sur des années 